# Providentia

## Alpaca Market

Alpaca Markets offers trading that is quickly gaining popularity because it does not charge a commission for selling or buying stocks. It is focused on API trading and allows investors to easily use artificial intelligence, trading bots and algorithms in their trading and investing activities.

Form W8B:

https://www.irs.gov/pub/irs-pdf/fw8ben.pdf

## FinRL

FinRL is the open source library for practitioners. To efficiently automate trading, AI4Finance provides this educational resource and makes it easier to learn about deep reinforcement learning (DRL) in quantitative finance.


In [14]:
!pip install -r requirements.txt

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /private/var/folders/2s/3s8clx5j7r79361kj3f7y_5c0000gn/T/pip-req-build-zxvkqte9
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /private/var/folders/2s/3s8clx5j7r79361kj3f7y_5c0000gn/T/pip-req-build-zxvkqte9
  Cloning https://github.com/quantopian/pyfolio.git to /private/var/folders/2s/3s8clx5j7r79361kj3f7y_5c0000gn/T/pip-install-f48hm1zq/pyfolio_04aaff3b5fb640ec88c0819896b5bd89
  Running command git clone -q https://github.com/quantopian/pyfolio.git /private/var/folders/2s/3s8clx5j7r79361kj3f7y_5c0000gn/T/pip-install-f48hm1zq/pyfolio_04aaff3b5fb640ec88c0819896b5bd89


In [21]:
from alpaca_trade_api.rest import REST,TimeFrame
import alpaca_trade_api as tradeapi


In [26]:
api = REST('AKCLP03YEDI2RURGO9WF', '2N7SbDGOgzknPKs1f5QanvQkp7R6qySjenzC7o8a')

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2021-06-08 08:00:00+00:00,126.10,126.10,125.96,126.00,2923,53,126.030181
2021-06-08 08:01:00+00:00,125.97,125.98,125.96,125.98,896,16,125.974330
2021-06-08 08:02:00+00:00,126.00,126.04,125.98,126.04,1315,29,126.000966
2021-06-08 08:03:00+00:00,126.09,126.10,126.09,126.10,1381,20,126.093816
2021-06-08 08:04:00+00:00,126.10,126.16,126.10,126.16,1890,66,126.114196


In [38]:
class AlpacaDownloader:
    """Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API

    """

    def __init__(self, start_date: str, end_date: str, ticker_list: list, api_key: str, secret_key: str):
        # AKCLP03YEDI2RURGO9WF
        # 2N7SbDGOgzknPKs1f5QanvQkp7R6qySjenzC7o8a
        self.start_date = start_date
        self.end_date = end_date
        self.ticker_list = ticker_list
        self.api = REST(api_key, secret_key)


    def fetch_data(self) -> pd.DataFrame:
        """Fetches data from Yahoo API
        Parameters
        ----------

        Returns
        -------
        `pd.DataFrame`
            7 columns: A date, open, high, low, close, volume and tick symbol
            for the specified stock ticker
        """
        # Download and save the data in a pandas DataFrame:
        data_df = pd.DataFrame()
        for tic in self.ticker_list:
            temp_df = self.api.get_bars(tic, TimeFrame.Day, self.start_date , self.end_date, adjustment='raw').df
            temp_df["tic"] = tic
            data_df = data_df.append(temp_df)
        # reset the index, we want to use numbers as index instead of dates
        data_df = data_df.reset_index()
        try:
            # convert the column names to standardized names
            data_df.columns = [
                "date",
                "open",
                "high",
                "low",
                "close",
                "volume",
                "trade_count",
                "vwap",
                'tic'
            ]
            # use adjusted close price instead of close price
            #data_df["close"] = data_df["adjcp"]
            # drop the adjusted close price column
            data_df = data_df.drop("trade_count", 1)
            data_df = data_df.drop("vwap", 1)

        except NotImplementedError:
            print("the features are not supported currently")
        # create day of the week column (monday = 0)
        data_df["day"] = data_df["date"].dt.dayofweek
        # convert date to standard string format, easy to filter
        data_df["date"] = data_df.date.apply(lambda x: x.strftime("%Y-%m-%d"))
        # drop missing data
        data_df = data_df.dropna()
        data_df = data_df.reset_index(drop=True)
        print("Shape of DataFrame: ", data_df.shape)
        # print("Display DataFrame: ", data_df.head())

        data_df = data_df.sort_values(by=['date','tic']).reset_index(drop=True)

        return data_df

    def select_equal_rows_stock(self, df):
        df_check = df.tic.value_counts()
        df_check = pd.DataFrame(df_check).reset_index()
        df_check.columns = ["tic", "counts"]
        mean_df = df_check.counts.mean()
        equal_list = list(df.tic.value_counts() >= mean_df)
        names = df.tic.value_counts().index
        select_stocks_list = list(names[equal_list])
        df = df[df.tic.isin(select_stocks_list)]
        return df


In [41]:
DOW_TICKERS = ['AAPL', 'MSFT', 'JPM', 'V', 'RTX', 'PG', 'GS', 'NKE', 'DIS', 'AXP', 'HD',
               'INTC', 'WMT', 'IBM', 'MRK', 'UNH', 'KO', 'CAT', 'TRV', 'JNJ', 'CVX', 'MCD',
               'VZ', 'CSCO', 'XOM', 'BA', 'MMM', 'PFE', 'WBA', 'DD']

In [40]:
AlpacaDownloader(start_date = '2008-01-01',
                 end_date = '2021-08-11',
                 ticker_list = DOW_TICKERS,
                 api_key = 'AKCLP03YEDI2RURGO9WF',
                 secret_key = '2N7SbDGOgzknPKs1f5QanvQkp7R6qySjenzC7o8a').fetch_data()

Shape of DataFrame:  (41928, 8)


,date,open,high,low,close,volume,tic,day
0,2015-12-01,118.73,118.810,116.8600,117.34,34852374,AAPL,1
1,2015-12-01,71.92,72.445,71.5700,72.20,4193461,AXP,1
2,2015-12-01,147.30,149.400,146.4700,147.74,3444346,BA,1
3,2015-12-01,72.42,72.640,70.8500,71.56,6578365,CAT,1
4,2015-12-01,27.21,27.650,27.2000,27.57,31406279,CSCO,1
...,...,...,...,...,...,...,...,...
41923,2021-08-11,238.17,238.360,234.5800,235.06,7359738,V,2
41924,2021-08-11,55.62,55.845,55.5200,55.57,11862224,VZ,2
41925,2021-08-11,48.63,49.845,48.4601,49.72,7750023,WBA,2
41926,2021-08-11,149.56,151.580,149.1600,150.01,13810489,WMT,2


In [23]:

api = tradeapi.REST('AKCLP03YEDI2RURGO9WF', '2N7SbDGOgzknPKs1f5QanvQkp7R6qySjenzC7o8a')
account = api.get_account()
account.status


'ACTION_REQUIRED'

In [25]:
# Ejecutar ordenes 

api.submit_order(
    symbol='SPY',
    side='buy',
    type='market',
    qty='1',
    time_in_force='day',
    order_class='bracket',
    take_profit=dict(
        limit_price='305.0',
    ),
    stop_loss=dict(
        stop_price='295.5',
        limit_price='295.5',
    )
)

APIError: account is not authorized to trade